 ## **[파이썬 코딩을 통한 텍스트 데이터 수집방법]**
 
 
 ### 1. BS(Beautiful Soup)을 활용한 데이터 수집 ( 네이버 블로그 )  
 
 
 ### 2. selenium을 활용한 데이터 수집 (네이버 쇼핑후기) 
 
 
 
 
   

# 1. BS를 활용한 데이터 수집 (네이버 블로그)

## 1_1. 필요한 라이브러리 설치

In [53]:
!pip install pandas   
!pip install xlsxwriter    
!pip install BS4   
!pip install wordcloud    

  Created wheel for BS4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=ccbad097c52e5a5715f7e562e70fcb5947b5f68905fca94f761c9a2bbd24340c
  Stored in directory: c:\users\yti18\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built BS4


In [1]:
import re  
import json  
import math  
import requests  
import urllib.request 
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup   
import pandas as pd
import xlsxwriter
from tqdm import tqdm  

## 1_2. 네이버 개발자 사이트에서 ID와 Secret 얻기
네이버에서 제공하는 API를 이용하려면 네이버 개발자 사이트 https://developers.naver.com 에서 어플리케이션 등록 후, Client ID는 naver_clinet_id에 넣고, Client Secret은 naver_client_secret에 넣는다.

별도의 id와 비밀번호를 받을 필요없이 아래의 ID와 비밀번호를 그대로 사용하면 된다.

## 1_3. 네이버 블로그 갯수 가져오기


In [3]:
def get_blog_count(query, display): #검색어 해당하는 블로그 전체 개수 가져오기
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query
    request = urllib.request.Request(search_url) 

    request.add_header("X-Naver-Client-Id", naver_client_id)  #헤더추가
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request) #실제 사이트에 해당하는 결과 요청
    response_code = response.getcode()  #반환되는 코드 확인

    if response_code is 200: # 서버가 요청을 제대로 처리한 경우
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))  # 네이버가 제공하는 JSON파일로 읽기

        print("Last build date: " + str(response_body_dict['lastBuildDate']))  # JSON 출력 중 해당 항목의 필드를 가져와 출력
        print("Total: " + str(response_body_dict['total']))
        print("Start: " + str(response_body_dict['start']))
        print("Display: " + str(response_body_dict['display']))

        if response_body_dict['total'] == 0:
            blog_count = 0
        else:
            blog_total = math.ceil(response_body_dict['total'] / int(display))

            if blog_total >= 1000:   # 검색결과를 최대 1000개까지 출력
                blog_count = 1000
            else:
                blog_count = blog_total

            print("Blog total: " + str(blog_total))
            print("Blog count: " + str(blog_count))

        return blog_count

## 1_4. 네이버 블로그 포스트 가져오기

In [3]:
def get_blog_post(query, display, start_index, sort):  # 검색된 블로그 내용 가져오기
    global no, df
    
    encode_query = urllib.parse.quote(query)  # 검색API에 요청할 변수로 query외에 display, start, sort 추가
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + \
                encode_query + "&display=" + str(display) + "&start=" + str(start_index) + "&sort=" + sort

    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", naver_client_id)  # 아이디와 비밀번호를 헤더에 추가
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))
        for item_index in range(0, len(response_body_dict['items'])):
            try:                              # item 필드 안에 있는 title 등의 필드에 대한 내용 가져오기
                remove_html_tag = re.compile('<.*?>') #HTML 태그 제거를 위한 정규표현식 함수 사용
                title = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['title'])
                link = response_body_dict['items'][item_index]['link'].replace("amp;", "")
                description = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']

                no += 1
                post_code = requests.get(link)
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text, 'lxml')

                blog_post_content_text = ""
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com" + mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text, 'lxml')
                    
                    for blog_post_content in blog_post_soup.find_all('div', class_='se-viewer'):
                        blog_post_content_text = blog_post_content.get_text()
                        
                    for blog_post_content in blog_post_soup.find_all('div', class_='se_doc_viewer'):
                        blog_post_content_text = blog_post_content.get_text()
 
                    for blog_post_content in blog_post_soup.select('div#postViewArea'):
                        blog_post_content_text = blog_post_content.get_text()

                df.loc[no] = [title, link, description, blogger_name, blogger_link, post_date, blog_post_content_text]
                print("#", end='')
                
            except:
                item_index += 1

## 1_5. 네이버 블로그 텍스트 수집 시작
- 약 6분~8분 소요

In [5]:
# query = "SK매직 가스레인지" 부분에서 원하는 검색어만 변경.

no = 0                 # 몇개의 포스트를 저장하였는지 세기 위한 index
query = "SK매직 청소기"   # 검색을 원하는 문자열로서 UTF-8로 인코딩한다.
display = 10           # 검색 결과 출력 건수 지정, 10(기본값),100(최대)
start = 1              # 검색 시작 위치로 최대 2000까지 가능
sort = "sim"           # 정렬 옵션: sim(유사도순, 기본값), date(날짜순)

# 블로그를 DataFrame에 저장
df = pd.DataFrame(columns=("Title", "Link", "Description", "Blogger Name", "Blogger Link", "Post Date", "Post Contents"))

blog_count = get_blog_count(query, display)
for start_index in range(start, blog_count + 1, display):
    get_blog_post(query, display, start_index, sort)

NameError: name 'get_blog_count' is not defined

## 1_6. 수집된 데이터 파일로 저장한 후 불러오기

### ** 코드파일들을 저장한 폴더 내에 data라는 이름의 하위 폴더를 미리 생성하여 향후 여러 데이터 파일들을 저장

In [6]:
# 수집된 블로그 텍스트 데이터를 분석용 csv 파일로 저장 

df.to_csv("./data/blog.csv", header=True, index=False, encoding='utf-8-sig') 


In [7]:
# 데이터를 읽기 위한 xslx 파일로 저장
writer = pd.ExcelWriter('./data/skm_blog.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1') # xslx 파일로 저장 
writer.close() # Pandas writer 객체 닫기

In [5]:
# 저장된 csv 파일 불러오고 확인하기

import pandas as pd
pd.set_option('display.max.colwidth', 100)
df_1 = pd.read_csv('./data/skm_blog.csv', encoding='utf-8-sig') 
df_1

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
0,물걸레 청소기 무선 추천 휴스톰-신일-SK매직...,https://blog.naver.com/autobahnkids?Redirect=Log&logNo=222543529401,물걸레 청소기 무선 추천 휴스톰-신일-SK매직-대우어플라이언스-라우쉬 [탑성비5] 벌써 날씨가 추워졌습니다. 최근엔 오랜 시간 문을 열어 환기를 시키기가 무척 까다로워졌는데...,내돈내산정보,https://blog.naver.com/autobahnkids,20211021.0,\n\n물걸레 청소기 무선 추천 휴스톰-신일-SK매직-대우어플라이언스-라우쉬 [탑성비5]벌써 날씨가 추워졌습니다. 최근엔 오랜 시간 문을 열어 환기를 시키기가 무척 까다로워...
1,"털빠짐 많은 대형견 필수품, 대용량 청소기 추천, SK매직...",https://blog.naver.com/szin93?Redirect=Log&logNo=222244834789,"영업용, 업소용 청소기라고 불리더라구요 저는 찾아보다가 SK매직 청소기로 구매했습니다 네이버에서 검색 후 129800원에 겟 요즘 일반 가정용 청소기 엄청 비싼데 이정도면 ...",vive,https://blog.naver.com/szin93,20210215.0,"\n\n\n\n\n\n\nReview\n\n\n\n\n털빠짐 많은 대형견 필수품, 대용량 청소기 추천, SK매직 공업용 청소기 \n\n\n\n\n\n섲인\n\n ・ \n20..."
2,조용하고 청소 잘되는 SK매직 듀얼스핀 무선 물걸레청소기...,https://blog.naver.com/minsoism?Redirect=Log&logNo=222205058585,"아...안돼 눈독들이지뫄 SK매직 무선 물걸레청소기 구입하고 나서도 걱정스러웠던 건 바로 소음이었어요. 먹잠먹잠하는 신생아가 있어서 소음에 민감해졌죠. 애 잘 때, 잠들려고...",민소와 내내 건강하소,https://blog.naver.com/minsoism,20210112.0,\n\n\n\n\n\n\n체험을해봤어요\n\n\n\n\n조용하고 청소 잘되는 SK매직 듀얼스핀 무선 물걸레청소기 명절선물로 추천! \n\n\n\n\n\n민소\n\n ・ \n...
3,오픈하우스 달꼬미아루의 장비포스팅 - (2) SK매직 물걸레청소기,https://blog.naver.com/aisuru0704?Redirect=Log&logNo=222147995330,뒤에 SK 매직 물걸레청소기를 사용해서 뽀송뽀송한 바닥을 관리하고 있습니다. 처음 인수하고 나서는 바닥이 너무 끈적끈적해서 입주자들의 컴플레인도 있어서 스팀청소기를 구입했어...,달꼬미아루 오픈하우스 응암,https://blog.naver.com/aisuru0704,20201118.0,\n\n\n\n\n\n\n오픈하우스 응암역점 이야기\n\n\n\n\n응암동 오픈하우스 달꼬미아루의 장비포스팅 - (2) SK매직 물걸레청소기 \n\n\n\n\n\n달꼬미아...
4,자동 물걸레 청소기 추천 SK매직 무선 듀얼스핀 물걸레...,https://blog.naver.com/jerajinbk?Redirect=Log&logNo=222225868786,SK매직 무선 듀얼스핀 물걸레 청소기 VCL-WA10 자동물걸레청소기 추천 글 / 사진 : 제라진BK #무선물걸레청소기 #물걸레청소기 #SK매직 #VCL-WA10 #듀얼스핀...,제라진BK의 제라진 블로그,https://blog.naver.com/jerajinbk,20210130.0,\n\n\n\n\n\n\nIT / 게임\n\n\n\n\n자동 물걸레 청소기 추천 SK매직 무선 듀얼스핀 물걸레 청소기 VCL-WA10 \n\n\n\n\n\n제라진BK\n\n...
...,...,...,...,...,...,...,...
555,SK매직올클린바이러스핏 살균 공기청정기라 특별해요.,https://blog.naver.com/makemage27?Redirect=Log&logNo=222294811263,아마 SK매직 올클린바이러스핏만의 독특한 기능이 아닐까 싶은데요. UV와 광촉매를 결합한 UV 아크 코어... 청소기 툴을 바꿔서 돌려도 먼지가 깔끔하게 안빠지고 면봉으로 ...,마눌꽁치's 요리 Cock!,https://blog.naver.com/makemage27,20210401.0,\n\n\n\n\n\n\n\n\n\n☞ 가전 디지털\n\n\n\n\nSK매직올클린바이러스핏 살균 공기청정기라 특별해요. \n\n\n\n\n\n마눌꽁치\n\n ・ \n202...
556,식기세척기 세제 sk매직DWA-7303D와 자연퐁스팀워시의 만남...,https://blog.naver.com/nikekkw?Redirect=Log&logNo=222282465722,로봇청소기는 남자1호 눈치보느냐고 아직 장바구니에서 잠자고 있고..(조만간 질러) 3달전쯤에 망고짱도 식세기이모님 #sk매직DWA-7303D 이오셨어요. 아주 사랑하는 이모...,똥꼬발랄한 나의 하루,https://blog.naver.com/nikekkw,20210321.0,\n\n\n\n\n\n\n\n\n\n내가 뜯어 볼께~상품리뷰\n\n\n\n\n식기세척기 세제 sk매직DWA-7303D와 자연퐁스팀워시의 만남 최강조합 실사용후기 \n\n\n...
557,[리뷰] SK매직 6인용 식기세척기(DWA1812) - 생활가전...,https://blog.naver.com/kmj290?Redirect=Log&logNo=222234279966,로봇청소기 이모들 덕분에 빨래와 청소부분은 큰 도움을 받고 있다. ▼건조기 신세계(가장 고마움... 여러 제품 중 후기가 좋았던 SK매직 6인용 식세기(DWA1812)! 때...,행복한 우리집,https://blog.naver.com/kmj290,20210206.0,\n\n\n\n\n\n\n리뷰\n\n\n\n\n[리뷰] SK매직 6인용 식기세척기(DWA1812) - 생활가전 삼대장의 완성! \n\n\n\n\n\n빈이맘\n\n ・ \n2...
558,[공유] SK매직 물세척 진공청소기를 4만원대로 구입할 수...,https://blog.naver.com/pbkim2?Redirect=Log&logNo=221391151039,청소기가 딱히 필요 없으면 모르겠지만... 이건 사야 하지 않을까요??!! 구입하러 가기 [링크] &lt;script type=&quot;text/javascript&quo...,"게임,애니,영화,그리고 잡다한 이야기",https://blog.naver.com/pbkim2,20181104.0,"\n\n 이번 십일절에 예약 구매 품목으로 나왔던데, 시중에서 7만원대가 최저가로 확인이 되는 제품이예요.그런데 이번에 특가로 49,900원이랍니다!!청소기가 딱히 필요 없..."


In [10]:
len(df_1)  # 파일이 담고 있는 전체 텍스트 개수 확인

560

In [11]:
df_1.head()  # 상위 5개 텍스트 확인

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
0,"털빠짐 많은 대형견 필수품, 대용량 청소기 추천, SK매직...",https://blog.naver.com/szin93?Redirect=Log&log...,"영업용, 업소용 청소기라고 불리더라구요 저는 찾아보다가 SK매직 청소기로 구매했습니...",vive,https://blog.naver.com/szin93,20210215.0,"\n\n\n\n\n\n\nReview\n\n\n\n\n털빠짐 많은 대형견 필수품, ..."
1,조용하고 청소 잘되는 SK매직 듀얼스핀 무선 물걸레청소기...,https://blog.naver.com/minsoism?Redirect=Log&l...,아...안돼 눈독들이지뫄 SK매직 무선 물걸레청소기 구입하고 나서도 걱정스러웠던 건...,민소와 내내 건강하소,https://blog.naver.com/minsoism,20210112.0,\n\n\n\n\n\n\n체험을해봤어요\n\n\n\n\n조용하고 청소 잘되는 SK매...
2,오픈하우스 달꼬미아루의 장비포스팅 - (2) SK매직 물걸레청소기,https://blog.naver.com/aisuru0704?Redirect=Log...,뒤에 SK 매직 물걸레청소기를 사용해서 뽀송뽀송한 바닥을 관리하고 있습니다. 처음 ...,달꼬미아루 오픈하우스 응암,https://blog.naver.com/aisuru0704,20201118.0,\n\n\n\n\n\n\n오픈하우스 응암역점 이야기\n\n\n\n\n응암동 오픈하우...
3,자동 물걸레 청소기 추천 SK매직 무선 듀얼스핀 물걸레...,https://blog.naver.com/jerajinbk?Redirect=Log&...,SK매직 무선 듀얼스핀 물걸레 청소기 VCL-WA10 자동물걸레청소기 추천 글 / ...,제라진BK의 제라진 블로그,https://blog.naver.com/jerajinbk,20210130.0,\n\n\n\n\n\n\nIT / 게임\n\n\n\n\n자동 물걸레 청소기 추천 S...
4,이마트 국민가격 물걸레 청소기 (SK매직물걸레청소기와 비교),https://blog.naver.com/onechickenleg?Redirect=...,#이마트물걸레청소기 #노브랜드물걸레청소기 #국민가격물걸레청소기 #물걸레청소기 #JJ...,REVIEWER,https://blog.naver.com/onechickenleg,20191228.0,\n\n\n\n\n\n\nSOMETHING NEW\n\n\n\n\n이마트 국민가격 ...


In [13]:
df_1.tail()   # 마지막 5개 텍스트 확인

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
555,SK 매직 미니온 식기세척기 (SK매직 미니식기세척기),https://blog.naver.com/kumseo91?Redirect=Log&l...,SK 미니온 식기세척기 몇 일 동안 말이 많아서 저도 고민하고 있었는데 SK매직에서...,소소한 일상,https://blog.naver.com/kumseo91,20200728.0,\n\n\n\n\n\n\nreview\n\n\n\n\nSK 매직 미니온 식기세척기 ...
556,[리뷰] SK매직 6인용 식기세척기(DWA1812) - 생활가전...,https://blog.naver.com/kmj290?Redirect=Log&log...,로봇청소기 이모들 덕분에 빨래와 청소부분은 큰 도움을 받고 있다. ▼건조기 신세계(...,행복한 우리집,https://blog.naver.com/kmj290,20210206.0,\n\n\n\n\n\n\n리뷰\n\n\n\n\n[리뷰] SK매직 6인용 식기세척기(...
557,[가전] SK매직 식기세척기(무설치형) 설치 및 사용 리뷰...,https://blog.naver.com/dotorimoa?Redirect=Log&...,있는 SK매직의 DWA-2800P 모델을 구입했습니다. (사이즈: 420X435X4...,보람찬 일상 이야기★,https://blog.naver.com/dotorimoa,20210414.0,\n\n\n\n\n\n\n생활\n\n\n\n\n[가전] SK매직 식기세척기(무설치형...
558,SK매직 진공청소기 VCL-616 : 가성비 좋음,https://blog.naver.com/bb2da?Redirect=Log&logN...,다이슨 무선청소기 덕에 비슷한 무선제품이 많이 나오지만 배터리 문제도 있고 무선청소...,봉봉이와 함께 쓰는 블로그,https://blog.naver.com/bb2da,20180113.0,\n\n\n\n\n\n\n\n\n상품 리뷰\n\n\n\n\n\n\n\n\n\n\nS...
559,SK매직 올클린 공기청정기 바이러스핏 체험단을 모집해요.,https://blog.naver.com/kirasjy?Redirect=Log&lo...,SK매직 올클린 공기청정기 바이러스핏 체험단을 모집해요. 함께 이벤트에 참여해보시라...,매일이기쁨_시골 육아 이야기,https://blog.naver.com/kirasjy,20210315.0,\n\n\n\n\n\n\n리뷰.공유\n\n\n\n\nSK매직 올클린 공기청정기 바이...


# 2. selenium을 활용한 네이버 쇼핑후기 수집하기 

### *** chromedriver가 pc에 없는 경우 미리 다운 받아서 코드가 있는 동일한 폴더에 설치할 것 ***

   1. 크롬드라이버 설치하기 전에 본인PC의 크롬 브라우저 버전 확인하기
     ... 크롬 브라우저 우측 상단의 크롬맞춤설정 및 제어 클릭 
         --> 좌측 중간에 chrome 정보 
         --> 본인의 크롬정보 확인 (예: 93.0.4577.82)
     
    2.아래 사이트에서 본인의 크롬버전에 맞는 크롬 드라이버를 다운받아 압출파일을 풀고 바탕화면에 설치하기   
	https://sites.google.com/a/chromium.org/chromedriver/downloads


    3. 본인의 파이썬 코드와 같은 폴더(보기: PTM2021)로 옮겨 놓기.
    
    

## 2_1. 필요한 라이브러리 설치

In [8]:
!pip install Selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [12]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
from time import sleep
from urllib.parse import quote_plus
import requests
import re
import pandas as pd
import numpy as np
import os

## 2_2. 쇼핑후기 텍스트 데이터를 담는 데이터 프레임 생성 

In [13]:
def add_dataframe(name, category,date, title, reviews, stars, cnt):  # 크롤링한 데이터 데이터 프레임 생성하는 함수 
    
    df1=pd.DataFrame(columns=['type', 'category', 'date', 'title', 'review', 'star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name, category, date[i], titles[i], reviews[i], stars[i]]
            i+=1
            n+=1
    else:
        df1.loc[n]=[name, category, 'null' , 'null', 'null', 'null']
        n+=1    
    return df1

## 2_2. 검색할 키워드 입력  

- 다음의 주소를 클릭하여 네이버 쇼핑몰에 직접 접속한 후 핵심어(브랜드 제품명)를 입력하여 검색

 https://shopping.naver.com/   



## 2_3. 검색할 페이지 찾아서 쇼핑후기 텍스트 수집하기



1) 네이버 쇼핑몰 페이지에서 검색할 검색어 입력 (보기: sk매직 가스레인지)  

2) 검색된 상품 리스트를 '리뷰 많은 순'으로 정렬

3) 우측에 다수의 쇼핑몰이 '쇼핑몰별 최저가'로 제시되어 있는 모델들 선정 (쇼핑몰이 하나만 제시된 모델은 해당 쇼핑몰로 이동)

4) 복수의 쇼핑몰이 제시된 모델 중 리뷰가 1,000개 남짓한 제품 선정 (리뷰가 너무 많으면 별점이 편중되어 수집) 

5) 최종 선택된 모델이 있는 사이트에서 쇼핌몰 리뷰 섹션을 클릭

6) 리뷰 섹션의 주소(url) 복사하여 ns_address = "..."에 붙이기

7) 별도의 창이 열려 쇼핑후기 사이트가 자동으로 구동되다가 없어지는 동안 스크롤 건드리지 말고 기다릴 것. 


In [16]:
# 아래 코드 중에서 수정할 부분
# 1) name=['sk매직 가스레인지'] 에서 검색어 수정
# 2) ns_address = "...." 부분에 수집할 쇼핑몰 리뷰 페이지 url 복사해서 붙여넣기


name=['sk매직 청소기']   # 검색할 브랜드 및 제품명 이름 넣기(**)
category=['별점']


# 수집할 '쇼핑몰리뷰' 페이지 url 복사해서 붙여넣기 (**)
ns_address = "https://search.shopping.naver.com/catalog/12359082902?query=sk%EB%A7%A4%EC%A7%81%20%EC%B2%AD%EC%86%8C%EA%B8%B0&NaPm=ct%3Dkupc3mk8%7Cci%3D9796d3175509888769b8aaa29e25bdcba6cdc709%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Da0e3355e037ce6562e8727b45f52db086cd47d12"
shoppingmall_review_css="#snb > ul > li:nth-child(2) > a"

header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/realy/Desktop/2021-2/텍스트마이닝/chromedriver.exe') # chromedriver.exe가 있는 경로를 입력해주세요. (**)
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기 & # 리뷰 가져오기
element = d.find_element_by_css_selector(shoppingmall_review_css).click() #스크롤 건드리면 안됨
sleep(2)

name_=name[0]
category_=category[0]
dates, reviews, titles, stars = [], [], [], []
cnt=1
page=1

while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)

    while True: # 한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star = d.find_element_by_css_selector("#section_review > ul > li:nth-child("+str(j)+") > div.reviewItems_etc_area__2P8i3 > span.reviewItems_average__16Ya-").text
            stars.append(star)
            date = d.find_element_by_css_selector("#section_review > ul > li:nth-child("+str(j)+") > div.reviewItems_etc_area__2P8i3 > span:nth-child(4)").text
            dates.append(date)
            title = d.find_element_by_css_selector("#section_review > ul > li:nth-child("+str(j)+")> div.reviewItems_review__1eF8A > div > em").text
            titles.append(title)
            review = d.find_element_by_css_selector("#section_review > ul > li:nth-child("+str(j)+") > div.reviewItems_review__1eF8A> div > p.reviewItems_text__XIsTc").text
            reviews.append(review)
            j+=1
            print(cnt, star, date, title, "\n") # 리뷰index, 별점, 제목 출력
            cnt+=1
        except:  
            break # 한페이지에 20개 리뷰 마지막 리뷰에서 error 발생 후, 종료
            
    sleep(2)

    if page < 11: # 11 page까지 크롤링 됨 다음으로 넘어가야하기 떄문에 ! 숫자 변경하지 마세요! 
        try: # 리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element_by_css_selector("#section_review > div.pagination_pagination__2M9a4 > a:nth-child("+str(page)+")").click()
        except: 
            break # 리뷰의 마지막 페이지에서 process 종료 

    else:
        try: # page 11 부터 
            page+=1
            if page%10==0:
                next_page=d.find_element_by_css_selector('#section_review > div.pagination_pagination__2M9a4 > a.pagination_next__3ycRH').click()
                
            if page == 51: # 크롤링 원하는 페이지 +1 적용해서 변경하세요 (**)
                break
        except:
            break

d.close() # 크롤링이 끝나면 자동으로 창이 닫힘

C:\Users\realy\anaconda3\envs\py37\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


페이지 1 

1 평점5 19.07.23. 놀랍네요 

2 평점5 20.11.19. 무선 물걸레 사용은 처음이고 다른 제품 사용을 안해 봐서 비교 하기는 어렵지만 조립 부터 완충 후 첫 사... 

3 평점5 21.07.17. 배송은 소소. 포장은 박스를 누군가 개봉한 흔적이 살짝 보이는데 신경 안 씁니다.다만 엄마가 너무만족해 하십니다.물건은 좋습니다.청소를 해보고 후기를 올리는데 물기를 짠 다음 청소 

4 평점5 20.11.14. 즌작살껄~~ 

5 평점5 19.07.24. SK매직스핀업 무선물걸레청소기 VCL-W10B 패드2장추가 

6 평점5 19.08.09. 처음 사본 물 걸레 청소기 사용해 본 후기 

7 평점5 19.10.31. 할인 받아 저렴하게 8만원 가량에 구매했습니다.
생각보... 

8 평점5 19.07.06. 금요일날 도착 후 오늘 아침에 개봉 해서
조립 후 제품... 

9 평점5 19.08.24. 배송은 수요일 주문해서 금요일 오후에 받앗고, 상품과 구성품 하자없이 잘 왔습니다. 오자마자 신나서 사용해 봤는데 만족합니다. 주방바닥에 얼룩들과 기름때들도 그냥 여유있게 한자리에 

10 평점5 20.02.29. 무선이라 편한 건 두 말 할 것도 없고, 여느 물걸레청... 

11 평점3 19.08.13. 장점은 손잡이 떨어질때 충격완화쿠션 단점 어두운곳 잘보... 

12 평점5 19.08.25. 부모님 선물 해드렸어요! 검색해보니 타사 제품보다 이 제품이 힘도 덜 들고 훨씬 잘 닦인다고 해서 최종 구매했는데, 확실히 타사제품에 비해 안정적이고 힘이 덜든다고 하시네요! 충전 

13 평점5 19.01.17. sk매직 무선물걸레청소기 

14 평점5 19.10.28. 친구에게 선물함. 매우 좋아함. 소음이 거의 없어서 밤에도 사용가능하다고. 집에 놀러오신 시어머니께서도 보시고 같은 걸 주문했다 함. 역시 물청소기는 기존 청소기와 결합해 쓰는 것 

15 평점4 19.08.17. 두번째구매인데요 1년 쓰니까 못쓰게되네요 6개월 지나... 

페이지 5 

81 평점5 18.11.09. 청소하는게 즐거워졌어요! 청소해야지해야지 하면서도 하기 싫은 이유는 힘들고 귀찮아서죠..근데 SK매직 스핀업 사고나서 청소하는게 힘들고 귀찮지가 않아졌어요!!허리에 무리도 안가고 

82 평점5 18.11.12. 이제 청소할때 허리가 안아파요 ㅎㅎ 물걸레질하려고 바닥쪼그려닦다보면 무릎,허리 아파고생이었는데이 청소기사고나서 청소시간도줄고 너무편리해졌어요손으로 닦는것보다도 훨씬잘닦이고 디자인도 

83 평점5 18.11.13. 집들이선물 친구네 집들이 선물로 이거 콕 찝어사달라하길래 구입해줬습니다 ㅋㅋ다른집에 있는거보고 마음에둘어서 사달라했다네요 혼자 잘서있어서 공간차지도 적고 ㅋㅋ 줄도 없고 깔끔한 디 

84 평점5 20.12.03. 가볍고 손잡이도 긴편이라 안정적인 느낌이들고청소할때 힘이 들지 않아 좋네요배터리 잔량도 본체에서 바로 볼 수 있게끔 되어있어서 확인하기도 편해요더 사용해봐야 알겠지만 일단은 맘에 

85 평점5 21.08.16. 배터리 약해지는거 안느껴지고 잘쓰고 있어요 

86 평점5 21.06.05. 배송이 빠르고 기존 무선청소기에 물걸레키트 사용하는것보다 헤드모터 돌아가는 힘이 더 좋습니다. 전면부에 LED램프가 있어 먼지가 있는지 확인이 용이합니다 

87 평점5 21.06.10. 일단 배송이 빨라좋네요.써보고 다시 리뷰 남길께요..^^ 

88 평점4 21.05.30. 처음엔 무거워서 조종하기 어려웠는데 익숙해졌더니 편해요~~ 

89 평점5 21.09.20. 사용이 편리하지만 손잡이 부분이 약해서 두번째사용때 부 

90 평점5 21.05.10. 이거 완전 물건이네요다른 유명 브랜드 제품도 써봤는데 이게 훨씬 좋아요!완전 슬림해서 모퉁이 공간에 딱 들어가고 ㅎㅎ 

91 평점5 21.09.07. 전에 쓰던것보다 크고 잘나가네요.. 만족합니다 

92 평점5 21.09.07. 힘 안들이고 닦을수있어서 좋아요 

93 평점4 21.08.11. 좋아요 

94 평점5 21.08.24. 업체에서 잘

271 평점5 20.05.09. 저렴히 구매했어요!! 불이들어와 

272 평점5 20.05.24. sk매직 물걸레 청소기 

273 평점5 20.05.26. 잘 작동하고 배송도 잘 왔습니다~ 

274 평점5 20.05.30. 짱짱하네요. 잘 배송되었습니다. 

275 평점4 20.05.01. 배송 빠르게왔습니다 

276 평점5 20.05.21. 벌써 세번째 구매해서 사용하고 있는데 좋아요. 

277 평점5 20.01.09. 마음에 들어하네영 좋아용 많이파세욥!!!!ㅎㅎ 

278 평점5 20.01.12. 자동 물걸레 청소기 힘들이지 않고 청소할 수 있어 매우 만족했습니다. 

279 평점4 20.05.01. 좋아요 

280 평점4 20.01.13. 서비스 센터가 지역별로 다 있어서 강점인듯합니다 

페이지 15 

281 평점5 20.03.21. 두번째쓰는에스케이ㅎ 

282 평점5 20.07.01. 물청소가 힘들었는데 덕분에 잘 사용하고 있습니다 

283 평점5 20.03.25. 아주 잘됩니다. 좋습니다. 

284 평점3 20.02.21. 배송 빠르고 좋아요.포장도 이쁨 

285 평점5 20.02.23. 힘이 덜들어서 편해요 

286 평점5 20.05.29. 좋아요 

287 평점5 20.05.27. 가격대비 좋은상품입니다 

288 평점5 20.06.05. 좋아요 ㅎ... 

289 평점5 20.03.17. 좋아요 ~^^ 좋아요 

290 평점5 19.10.02. 무선 물걸레 청소기의 지존! 

291 평점5 20.05.09. 저렴히 구매했어요!! 불이들어와 

292 평점5 20.05.24. sk매직 물걸레 청소기 

293 평점5 20.05.26. 잘 작동하고 배송도 잘 왔습니다~ 

294 평점5 20.05.30. 짱짱하네요. 잘 배송되었습니다. 

295 평점4 20.05.01. 배송 빠르게왔습니다 

296 평점5 20.05.21. 벌써 세번째 구매해서 사용하고 있는데 좋아요. 

297 평점5 20.01.09. 마음에 들어하네영 

515 평점5 19.09.01. 좋네요.가격도 적당하고 좋습니다. 

516 평점5 19.09.12. 일단가벼워요 .소음도적은편이구 

517 평점5 19.05.18. 잘받았습니다 

518 평점4 19.05.14. 선물 했는데 만족하다 하네요 :) 

519 평점5 19.05.03. 적극 추천합니다! 

520 평점5 19.09.01. 선물했는데 너무 좋대요 

페이지 27 

521 평점5 19.06.06. 편해요 

522 평점5 19.09.30. 물뿌리는게 없는건 함정 

523 평점5 19.06.07. 조립했어요 

524 평점5 19.05.22. 잘 쓰겠습니다 

525 평점5 19.10.01. 부모님께 선물함 

526 평점4 19.09.16. 가성비 좋네요 

527 평점5 19.06.07. 좋습니다 

528 평점5 19.10.15. dfhryfdttdhtdjjtddjts 

529 평점5 19.05.22. 좋은상품입니다 

530 평점5 19.05.24. 제품 잘받았어요. 

531 평점5 19.09.02. 최고예요 

532 평점4 19.09.03. 속도는 느린것도 같지만 편하다고는 하네요. 

533 평점5 19.05.15. 만족 

534 평점5 19.05.21. 청소기 

535 평점5 19.09.01. 좋네요.가격도 적당하고 좋습니다. 

536 평점5 19.09.12. 일단가벼워요 .소음도적은편이구 

537 평점5 19.05.18. 잘받았습니다 

538 평점4 19.05.14. 선물 했는데 만족하다 하네요 :) 

539 평점5 19.05.03. 적극 추천합니다! 

540 평점5 19.09.01. 선물했는데 너무 좋대요 

페이지 28 

541 평점5 19.06.06. 편해요 

542 평점5 19.09.30. 물뿌리는게 없는건 함정 

543 평점5 19.06.07. 조립했어요 

544 평점5 19.05.22. 잘 쓰겠습니다 

545 평점5 19.10.01. 부모님께 선물함 

546 평점4 19.09.16. 가성비 좋

732 평점5 19.05.18. 바닥도 잘 닦이고진동소리도 작아요 

733 평점5 19.01.19. ㅎㅎ 

734 평점5 19.05.10. 조립쉽고 매우만족해요 

735 평점4 19.05.20. 아주 잘 사용 중입니다 

736 평점5 19.05.28. 배송빠르고 좋아요!! 

737 평점5 19.01.03. 무선물걸레청소기 

738 평점5 19.05.22. 청소기 좋내요 

739 평점4 19.01.22. 잘 받았어요 

740 평점5 19.01.26. 좋아요 잘 되요 어머니께서 좋아 하셔요...^^ 

페이지 38 

741 평점5 19.05.31. 좋습니다~~~ 

742 평점5 19.02.08. 물걸레청소기 짱 

743 평점5 19.02.19. 배송빠릅니다 

744 평점5 19.01.31. 좋아요 잘딱끼네요 ㅎㅎㅎ굿굿 

745 평점5 19.01.23. 완전 편해요! 엄마도 사드려야겠어요 ㅋㅋ청소 귀찮았는데 요새는 재밌게 청소하네요^^어둡고 좁은 공간도 편하게 할 수 있어요 

746 평점5 19.01.22. 진작살걸그랬어요 허리굽히지않고 걸레질이라니 너무편하고좋네요 잘닦여서 너무좋아요~ 

747 평점5 19.05.17. 아기 안고 한손으로도 물걸레청소 가능해요~ 넘 좋네요~^^ 

748 평점5 19.02.09. 잘 받았습니다 

749 평점5 19.05.23. 친구추천으루 엄마선물해드렷는데완전좋아하세용 

750 평점5 19.01.22. 사용해보니 정말 성능이 좋고 충전도 빠르고 좋았어요! 손쉽게 물걸레 청소할수 있어서 정말 신세계였어요^^ 

751 평점5 19.01.16. 깨끗하게 닦이고 사용하기 편리합니다. 

752 평점5 19.05.18. 바닥도 잘 닦이고진동소리도 작아요 

753 평점5 19.01.19. ㅎㅎ 

754 평점5 19.05.10. 조립쉽고 매우만족해요 

755 평점4 19.05.20. 아주 잘 사용 중입니다 

756 평점5 19.05.28. 배송빠르고 좋아요!! 

757 평점5 19.01.03. 무선물걸레

888 평점5 19.02.19. 개키우다보니 개털이 감당 안됐는데 청소가 훨편해졌네요 착착 잡혀요 

889 평점5 19.02.25. 물자국이 안남으며 편하게 청소할 수 있어요 ㅎㅎ 시간단축도 되요~ 

890 평점5 19.02.25. 헤드부분이 무게가 있지만 손목에 부담없이 물걸레질 할 수 있어요~ 

891 평점5 19.02.15. 힘들지 않게 청소가 가능하고 소음이 작으며 물자국이 안남는다. 

892 평점5 19.01.18. 너무 만족해요 ! 배송도 빠르고, 제품도 가성비 최고입니다 ◡̈ 

893 평점5 19.01.21. 어머니선물로 드렸는데 너무좋아하셔서 다행입니다 잘쓸게요~ 

894 평점5 19.02.25. sk브랜드를 믿고 질렀오요 ㅎㅎ 확실히 소음이 없는거 같아요 

895 평점5 19.02.21. 소음도 적고 길이 조절도 마음대로 가능하여 매우 편합니다. 

896 평점5 19.02.18. 나름 메이커라 생각하고 샀는데 역시 이름값을 하네요~!! 

897 평점5 19.02.25. 슥슥 지나가면서 깨끗하게 닦이니 엄청 편합니다 ㅎㅎ 

898 평점5 19.02.15. 후기보고 구매, 가성비 갑인듯!!!!!청소가 편리함!ㅎㅎㅎ 무릎꿇고 걸레질 안해도 되니 좋음! 

899 평점5 19.02.25. 소음이라하기에도 애매하네요 ㅋㅋ 완전 조용..!! 

900 평점5 19.02.19. 찌든때 걱정없이 편하게 걸레질할수있어 좋네요 

페이지 46 

901 평점5 19.02.14. 사용하기에도 너무 편하고 좋은 것 같아요!! 그동안 다른 거 쓸 때 부드럽지 않아서 불편해서 손이 아팠는데 이제 그럴 일 없을 것 같아요~ 

902 평점5 19.03.01. 배송 무지 빠르구 톡톡으로 문의드렸는데 너무 친절하셔서 감동 ㅎㅎ! 잘사용할게욥 

903 평점5 19.02.19. 정말 좋아요 이 가격 주고 진짜 잘 산 거 같아요 

904 평점5 19.02.11. 적극추천합니다!!허리,손목 부담이 전혀 없네요아주 편하게 잘 쓰고있어요 

905 평점5 19.0

## 2_4. 수집된 데이터 저장하기

In [17]:
shop_df = add_dataframe(name_,category_,dates, title, reviews, stars, cnt)

shop_df

,type,category,date,title,review,star
1,sk매직 청소기,별점,19.07.23.,놀랍네요,원목마루라 스팀청소 후 뒤틀림이 발생해서 매일 마른걸레질과 간혹 물걸레를 사용했었는...,평점5
2,sk매직 청소기,별점,20.11.19.,무선 물걸레 사용은 처음이고 다른 제품 사용을 안해 봐서 비교 하기는 어렵지만 조립...,무선 물걸레 사용은 처음이고 다른 제품 사용을 안해 봐서 비교 하기는 어렵지만 조립...,평점5
3,sk매직 청소기,별점,21.07.17.,배송은 소소. 포장은 박스를 누군가 개봉한 흔적이 살짝 보이는데 신경 안 씁니다.다...,배송은 소소. 포장은 박스를 누군가 개봉한 흔적이 살짝 보이는데 신경 안 씁니다.\...,평점5
4,sk매직 청소기,별점,20.11.14.,즌작살껄~~,타사제품 2대써봤어요한대는 성능이 좋았지만 유선이라 30평대 넘어가면 줄 이리저리 ...,평점5
5,sk매직 청소기,별점,19.07.24.,SK매직스핀업 무선물걸레청소기 VCL-W10B 패드2장추가,헤드부분이 무게감이 있어 힘들이지 않고 걸레질이 아주양호하게 잘됨 32평형 청소 힘...,평점5
...,...,...,...,...,...,...
996,sk매직 청소기,별점,19.01.04.,아이들이 있어서 청소를 잘 못했는데 소음이 작아서 아이들 잘 때 하면 되서 좋네요,아이들이 있어서 청소를 잘 못했는데 소음이 작아서 아이들 잘 때 하면 되서 좋네요,평점4
997,sk매직 청소기,별점,18.12.31.,간단한 조작법으로 쉽게쉽게 청소가 끝나네요 ㅎㅎ,간단한 조작법으로 쉽게쉽게 청소가 끝나네요 ㅎㅎ,평점4
998,sk매직 청소기,별점,18.12.31.,구석구석 잘 닦여요 ㅎㅎ 손목에도 무리가 없습니당,구석구석 잘 닦여요 ㅎㅎ 손목에도 무리가 없습니당,평점4
999,sk매직 청소기,별점,19.01.04.,조명이 달려있어서 밤에 조용히 물걸레질을 할 수 있어서 좋네요^^,조명이 달려있어서 밤에 조용히 물걸레질을 할 수 있어서 좋네요^^,평점5


In [18]:
shop_df['date'] = shop_df['date'].str.replace(".","")

year ="20"

shop_df['date'] = year + shop_df['date']
shop_df['date']

C:\Users\realy\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


1       20190723
2       20201119
3       20210717
4       20201114
5       20190724
          ...   
996     20190104
997     20181231
998     20181231
999     20190104
1000    20190102
Name: date, Length: 1000, dtype: object

크롤링된 파일 저장하기

In [19]:
shop_df

,type,category,date,title,review,star
1,sk매직 청소기,별점,20190723,놀랍네요,원목마루라 스팀청소 후 뒤틀림이 발생해서 매일 마른걸레질과 간혹 물걸레를 사용했었는...,평점5
2,sk매직 청소기,별점,20201119,무선 물걸레 사용은 처음이고 다른 제품 사용을 안해 봐서 비교 하기는 어렵지만 조립...,무선 물걸레 사용은 처음이고 다른 제품 사용을 안해 봐서 비교 하기는 어렵지만 조립...,평점5
3,sk매직 청소기,별점,20210717,배송은 소소. 포장은 박스를 누군가 개봉한 흔적이 살짝 보이는데 신경 안 씁니다.다...,배송은 소소. 포장은 박스를 누군가 개봉한 흔적이 살짝 보이는데 신경 안 씁니다.\...,평점5
4,sk매직 청소기,별점,20201114,즌작살껄~~,타사제품 2대써봤어요한대는 성능이 좋았지만 유선이라 30평대 넘어가면 줄 이리저리 ...,평점5
5,sk매직 청소기,별점,20190724,SK매직스핀업 무선물걸레청소기 VCL-W10B 패드2장추가,헤드부분이 무게감이 있어 힘들이지 않고 걸레질이 아주양호하게 잘됨 32평형 청소 힘...,평점5
...,...,...,...,...,...,...
996,sk매직 청소기,별점,20190104,아이들이 있어서 청소를 잘 못했는데 소음이 작아서 아이들 잘 때 하면 되서 좋네요,아이들이 있어서 청소를 잘 못했는데 소음이 작아서 아이들 잘 때 하면 되서 좋네요,평점4
997,sk매직 청소기,별점,20181231,간단한 조작법으로 쉽게쉽게 청소가 끝나네요 ㅎㅎ,간단한 조작법으로 쉽게쉽게 청소가 끝나네요 ㅎㅎ,평점4
998,sk매직 청소기,별점,20181231,구석구석 잘 닦여요 ㅎㅎ 손목에도 무리가 없습니당,구석구석 잘 닦여요 ㅎㅎ 손목에도 무리가 없습니당,평점4
999,sk매직 청소기,별점,20190104,조명이 달려있어서 밤에 조용히 물걸레질을 할 수 있어서 좋네요^^,조명이 달려있어서 밤에 조용히 물걸레질을 할 수 있어서 좋네요^^,평점5


In [20]:
# 수집된 블로그 텍스트 데이터를 분석용 csv 파일로 저장 

shop_df.to_csv("./data/skm_shopping.csv", header=True, index=False, encoding='utf-8-sig') 


xslx 파일로 저장허기

In [21]:
shop_crawling_xlsx = pd.ExcelWriter('./data/skm_shopping.xlsx', engine='xlsxwriter')  

shop_df.to_excel(shop_crawling_xlsx, sheet_name='Sheet1') 

shop_crawling_xlsx.close()

## 2_5. 저장한 텍스트 데이터 불러오기

In [2]:
import pandas as pd

df_2 = pd.read_csv('./data/skm_blog.csv', encoding='utf-8-sig') # csv 파일 불러오기

In [3]:
len(df_2)

560

In [4]:
df_2.head()

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
0,물걸레 청소기 무선 추천 휴스톰-신일-SK매직...,https://blog.naver.com/autobahnkids?Redirect=L...,물걸레 청소기 무선 추천 휴스톰-신일-SK매직-대우어플라이언스-라우쉬 [탑성비5] ...,내돈내산정보,https://blog.naver.com/autobahnkids,20211021.0,\n\n물걸레 청소기 무선 추천 휴스톰-신일-SK매직-대우어플라이언스-라우쉬 [탑성...
1,"털빠짐 많은 대형견 필수품, 대용량 청소기 추천, SK매직...",https://blog.naver.com/szin93?Redirect=Log&log...,"영업용, 업소용 청소기라고 불리더라구요 저는 찾아보다가 SK매직 청소기로 구매했습니...",vive,https://blog.naver.com/szin93,20210215.0,"\n\n\n\n\n\n\nReview\n\n\n\n\n털빠짐 많은 대형견 필수품, ..."
2,조용하고 청소 잘되는 SK매직 듀얼스핀 무선 물걸레청소기...,https://blog.naver.com/minsoism?Redirect=Log&l...,아...안돼 눈독들이지뫄 SK매직 무선 물걸레청소기 구입하고 나서도 걱정스러웠던 건...,민소와 내내 건강하소,https://blog.naver.com/minsoism,20210112.0,\n\n\n\n\n\n\n체험을해봤어요\n\n\n\n\n조용하고 청소 잘되는 SK매...
3,오픈하우스 달꼬미아루의 장비포스팅 - (2) SK매직 물걸레청소기,https://blog.naver.com/aisuru0704?Redirect=Log...,뒤에 SK 매직 물걸레청소기를 사용해서 뽀송뽀송한 바닥을 관리하고 있습니다. 처음 ...,달꼬미아루 오픈하우스 응암,https://blog.naver.com/aisuru0704,20201118.0,\n\n\n\n\n\n\n오픈하우스 응암역점 이야기\n\n\n\n\n응암동 오픈하우...
4,자동 물걸레 청소기 추천 SK매직 무선 듀얼스핀 물걸레...,https://blog.naver.com/jerajinbk?Redirect=Log&...,SK매직 무선 듀얼스핀 물걸레 청소기 VCL-WA10 자동물걸레청소기 추천 글 / ...,제라진BK의 제라진 블로그,https://blog.naver.com/jerajinbk,20210130.0,\n\n\n\n\n\n\nIT / 게임\n\n\n\n\n자동 물걸레 청소기 추천 S...


In [5]:
df_2.tail() # 뒤에서 0~4번째 행 출력

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
555,SK매직올클린바이러스핏 살균 공기청정기라 특별해요.,https://blog.naver.com/makemage27?Redirect=Log...,아마 SK매직 올클린바이러스핏만의 독특한 기능이 아닐까 싶은데요. UV와 광촉매를 ...,마눌꽁치's 요리 Cock!,https://blog.naver.com/makemage27,20210401.0,\n\n\n\n\n\n\n\n\n\n☞ 가전 디지털\n\n\n\n\nSK매직올클린바...
556,식기세척기 세제 sk매직DWA-7303D와 자연퐁스팀워시의 만남...,https://blog.naver.com/nikekkw?Redirect=Log&lo...,로봇청소기는 남자1호 눈치보느냐고 아직 장바구니에서 잠자고 있고..(조만간 질러) ...,똥꼬발랄한 나의 하루,https://blog.naver.com/nikekkw,20210321.0,\n\n\n\n\n\n\n\n\n\n내가 뜯어 볼께~상품리뷰\n\n\n\n\n식기세...
557,[리뷰] SK매직 6인용 식기세척기(DWA1812) - 생활가전...,https://blog.naver.com/kmj290?Redirect=Log&log...,로봇청소기 이모들 덕분에 빨래와 청소부분은 큰 도움을 받고 있다. ▼건조기 신세계(...,행복한 우리집,https://blog.naver.com/kmj290,20210206.0,\n\n\n\n\n\n\n리뷰\n\n\n\n\n[리뷰] SK매직 6인용 식기세척기(...
558,[공유] SK매직 물세척 진공청소기를 4만원대로 구입할 수...,https://blog.naver.com/pbkim2?Redirect=Log&log...,청소기가 딱히 필요 없으면 모르겠지만... 이건 사야 하지 않을까요??!! 구입하러...,"게임,애니,영화,그리고 잡다한 이야기",https://blog.naver.com/pbkim2,20181104.0,"\n\n 이번 십일절에 예약 구매 품목으로 나왔던데, 시중에서 7만원대가 최저가로 ..."
559,"SK매직 코어360 공기청정기, 강력하고 실용적이야!",https://blog.naver.com/newgisadan?Redirect=Log...,"에어컨, 공기청정기, 의류관리기, 건조기, 무선청소기, 로봇청소기 등 환경 및 생활...",릴라킹의 스마트하고 행복한 삶!,https://blog.naver.com/newgisadan,20200727.0,\n\n\n\n\n\n\n\n\n\n생활가전\n\n\n\n\nSK매직 코어360 공...
